In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
4,Hamilton 'grateful to be alive' after returnin...,https://www.espn.com/f1/story/_/id/30510811/ha...,"Sun, 13 Dec 2020 16:09:33 GMT",2020-12-13 16:09:33+00:00
0,First trucks with Covid-19 vaccine roll out of...,https://www.nbcnews.com/news/us-news/first-tru...,"Sun, 13 Dec 2020 15:27:00 GMT",2020-12-13 15:27:00+00:00
5,Influencers are using their clout and getting ...,https://www.cnn.com/2020/12/13/us/covid-influe...,"Sun, 13 Dec 2020 15:26:00 GMT",2020-12-13 15:26:00+00:00
2,COVID-19 Daily Update 12-13-2020 West Virgini...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sun, 13 Dec 2020 15:19:21 GMT",2020-12-13 15:19:21+00:00
3,"Maine CDC reports 299 COVID cases, no new deat...",https://www.pressherald.com/2020/12/13/maine-c...,"Sun, 13 Dec 2020 14:48:00 GMT",2020-12-13 14:48:00+00:00
8,Why These Two Epidemiologists Stopped Working ...,https://www.npr.org/2020/12/13/945989963/why-t...,"Sun, 13 Dec 2020 13:00:00 GMT",2020-12-13 13:00:00+00:00
1,COVID Vaccine Volunteer Traces His Motivation ...,https://www.npr.org/sections/health-shots/2020...,"Sun, 13 Dec 2020 12:00:25 GMT",2020-12-13 12:00:25+00:00
6,California could allow mass evictions to begin...,https://www.theguardian.com/us-news/2020/dec/1...,"Sun, 13 Dec 2020 11:00:00 GMT",2020-12-13 11:00:00+00:00
9,"UPDATE: Sanofi, GSK say COVID-19 shot won't be...",https://www.nbcnews.com/news/obituaries/update...,"Sat, 12 Dec 2020 21:31:00 GMT",2020-12-12 21:31:00+00:00
7,"Wisconsin crosses milestone of 4,000 deaths fr...",https://www.wsaw.com/2020/12/12/wisconsin-cros...,"Sat, 12 Dec 2020 20:20:00 GMT",2020-12-12 20:20:00+00:00
